In [1]:
import json
import os

In [2]:
def ParsingJson(textname, filename):
    with open("./data/raw/sample/"+textname+"/"+filename+".json", "r", encoding="utf8") as f:
        contents = f.read()
        file = json.loads(contents)
    keyJson = []
    if textname=="chat":
        keyJson = "utterances"
    elif textname=="news":
        keyJson = "sentences"
    
    countSentences = len(file[keyJson])
    
    result_list = []
    
    for k in range(countSentences):
        sentence = file[key][k]["text"]

        len_timex3 = len(file[key][k]["timex3"])
        len_event = len(file[key][k]["event"])
        info_list = [[] for i in range(len_event+len_timex3)]

        for i in range(len_event):
            event_s, event_e = file[key][k]["event"][i]["extent"]
            info_list[i].append(event_s)
            info_list[i].append(event_e)
            info_list[i].append('EVENT')

        for i in range(len_timex3):
            print("i:", i, "k:", k)
            timex3_s, timex3_e = file[key][k]["timex3"][i]["extent"]
            info_list[i+len_event].append(timex3_s)
            info_list[i+len_event].append(timex3_e)
            info_list[i+len_event].append(file[key][k]["timex3"][i]["type"])

        info_list.sort()

        named_entity = [0 for i in range(len(sentence))]
        for i in range(len(info_list)):
            start = info_list[i][0]
            end = info_list[i][1]
            netype = info_list[i][2]
            BorI = True
            for j in range(start, end):
                if BorI:
                    named_entity[j] = (netype, '-B')
                    BorI = False
                else:
                    named_entity[j] = (netype, '-I')

        label = []
        for i in range(len(sentence)):
            if named_entity[i]:
                label.append(named_entity[i][0]+named_entity[i][1])
            else:
                label.append('O')
            
        result = sentence + '\t'
        for i in range(len(label)):
            result += label[i] + ' '
        result = result.strip()
        result_list.append(result)
        
    return result_list

In [3]:
def saveTxt(textname, filename, res):
    f = open("./data/biotag/"+textname+"/"+filename+"txt", "w")
    for i in range(len(res)):
        f.write(res[i]+'\n')
    f.close()

In [4]:
# 파일이름 받아오기

text_name = ["chat", "news"]
file_list = []
for i in range(len(text_name)):
    file_list.append(os.listdir("./data/raw/sample/"+text_name[i]))

In [5]:
for i in range(len(file_list)):
    for j in range(len(file_list[i])):
        file_list[i][j] = file_list[i][j].split('.')[0]

In [6]:
print(file_list[0])

['대화06-5d4f682c', '대화07-5e938466', '대화05-4ec686e4', '대화03-2ca063cb', '대화01-01397ada', '대화11-bfc0f894', '대화10-bc57d852', '대화13-ed066712', '대화04-35c9421e', '대화12-d8679247', '대화02-1ac4b34b', '대화08-5f38abfb', '대화09-7dab8db4']


In [7]:
# json파일에서 데이터 추출하기

for i in range(len(file_list)):
    for j in range(len(file_list[i])):
        result = ParsingJson(text_name[i], file_list[i][j])
        saveTxt(text_name[i], file_list[i][j], result)

FileNotFoundError: [Errno 2] No such file or directory: './data/raw/sample/chat/대화06-5d4f682c.json'